In [11]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
from scipy import stats
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.model_selection import KFold, cross_val_score 

Read in mulitple csv files

In [12]:
# read in mulitple csv files
csv_files = ['data/gc_CGSS_RES_2024_M200_SEC01_fullgc_2024-02-27-11-47-30.csv',
             'data/gc_CGSS_RES_2024_M200_SEC02_fullgc_2024-02-27-11-46-45.csv',
             'data/gc_CGSS_RES_2024_M200_SEC04_fullgc_2024-02-27-11-45-43.csv',
             'data/gc_CGSS_RES_2024_M200_SEC05_fullgc_2024-02-27-11-41-46.csv' ,
             'data/gc_CGSS_RES_2024_M200_SEC06_fullgc_2024-02-27-11-40-29.csv',
             'data/gc_CGSS_RES_2024_M200_SEC07_fullgc_2024-02-27-11-39-47.csv',
             'data/gc_CGSS_RES_2024_M200_SEC08_fullgc_2024-02-27-11-38-45.csv',
             'data/gc_CGSS_RES_2024_M200_SEC09_fullgc_2024-02-27-11-35-51.csv',
             'data/gc_CGSS_RES_2024_M200_SEC10_fullgc_2024-02-27-11-34-33.csv',
             'data/gc_CGSS_RES_2024_M200_SEC11_fullgc_2024-02-27-11-35-06.csv', 
             'data/gc_CGSS_RES_2024_M200_SEC12_fullgc_2024-02-27-11-51-42.csv', 
             'data/gc_CGSS_RES_2024_M200_SEC13_fullgc_2024-02-27-11-30-36.csv', 
             'data/gc_CGSS_RES_2024_M200_SEC14_fullgc_2024-02-27-11-25-54.csv', 
             'data/gc_CGSS_RES_2024_M200_SEC15_fullgc_2024-02-27-11-53-00.csv',
             'data/gc_CGSS_RES_2024_M200_SEC16_fullgc_2024-02-27-11-53-56.csv',
             'data/gc_CGSS_RES_2024_M200_SEC17_fullgc_2024-02-27-11-13-09.csv',
             'data/gc_CGSS_RES_2024_M200_SEC18_fullgc_2024-02-27-11-11-21.csv',
             'data/gc_CGSS_RES_2024_M200_SEC19_fullgc_2024-02-27-11-10-45.csv'
            ]

# Initialize an empty list to store DataFrames
dataframes = []

# Read the first CSV file to get the column headers
initial_df = pd.read_csv(csv_files[0])
initial_columns = initial_df.columns

# Loop through the list of files
for file in csv_files:
    # Read the current CSV file into a DataFrame, ensuring it matches the initial columns
    df = pd.read_csv(file, usecols=lambda column: column in initial_columns).reindex(columns=initial_columns)
       
    # Drop rows where all cells are blank
    df.dropna(how='all', inplace=True)
    
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
m200_df = pd.concat(dataframes, ignore_index=True)

# Optionally, handle blanks in the compiled DataFrame as well

# Save the compiled DataFrame
m200_df.to_csv('data/m200_compiled_dataframe.csv', index=False)


In [13]:
# Assuming column 11 is 'M200B1 BB_exam_us' for US and column 12 is 'M200B1 BB_exam_ims' for international
# This will create a new column 'Combined_Scores' which has all scores, prioritizing US scores and filling in missing values with International scores.
m200_df['Combined_Scores'] = m200_df['M200B1 BB_exam_us'].combine_first(m200_df['M200B1 BB_exam_ims'])

# Save the updated DataFrame
m200_df.to_csv('data/m200_compiled_dataframe_scores_combined.csv', index=False)


In [14]:
m200_df.head()

,Last Name,First Name,Username,Learner ID,Last Access,Availability,Weighted Total,Total pts,M200P1 CTGL,M299A1 -div_mvmt_plan,M200B1 BB_exam_us,M200B1 BB_exam_ims,Combined_Scores
0,ACOSTA,MITCHELL,mitch.f.acosta,01C,2/26/2024 7:50,Yes,94.45000,285.50000,99,98.5,88.0,NaN,88.0
1,ALLEN,JUSTIN,justin.lee.allen,01A,2/17/2024 14:04,Yes,93.70000,283.00000,100,95.0,88.0,NaN,88.0
2,ALLISON,KEVIN,kevin.e.allison,01A,1/31/2024 11:29,Yes,95.09828,284.99429,93.99429,95.0,96.0,NaN,96.0
3,ALSUWAIDI,AHMED SAIF ABDULLA AHMED,alsuwaidi@hotmail.fr,01D,2/26/2024 13:59,Yes,89.10000,269.00000,93,92.0,NaN,84.0,84.0
4,ANDERSON,ALEX,alex.r.anderson,01C,2/26/2024 7:41,Yes,94.25000,283.50000,97,94.5,92.0,NaN,92.0


In [15]:
m200_df.tail()

,Last Name,First Name,Username,Learner ID,Last Access,Availability,Weighted Total,Total pts,M200P1 CTGL,M299A1 -div_mvmt_plan,M200B1 BB_exam_us,M200B1 BB_exam_ims,Combined_Scores
1036,WEAVER,BRIAN,brian.weaver9,19B,1/29/2024 9:08,Yes,94.99828,285.99429,93.99429,100.0,92.0,NaN,92.0
1037,WESTFALL,SAMUEL,samuel.g.westfall,19D,2/7/2024 11:20,Yes,90.29828,272.99429,93.99429,95.0,84.0,NaN,84.0
1038,WORKMAN,JASON,jason.workman3,19D,2/7/2024 8:20,Yes,88.79828,267.99429,93.99429,90.0,84.0,NaN,84.0
1039,WYATT,NICHOLAS,nicholas.j.wyatt,19D,1/25/2024 11:45,Yes,88.79828,267.99429,93.99429,90.0,84.0,NaN,84.0
1040,YOUNG,ROBERT,robert.d.young52,19C,2/11/2024 14:36,Yes,95.00000,286.00000,99.0,95.0,92.0,NaN,92.0


In [16]:
# Listing all column names in the DataFrame
column_names = m200_df.columns.tolist()
print(column_names)

['Last Name', 'First Name', 'Username', 'Learner ID', 'Last Access', 'Availability', 'Weighted Total', 'Total pts', 'M200P1 CTGL', 'M299A1 -div_mvmt_plan', 'M200B1 BB_exam_us', 'M200B1 BB_exam_ims', 'Combined_Scores']


In [17]:
# Assuming you want to keep columns named 
m200_df = m200_df['Last Name',
                   'First Name',
                   'Learner ID', 
                   'Weighted Total', 
                   'Total pts',
                   'M200P1 CTGL',
                   'M299A1 -div_mvmt_plan',
                   'M200B1 BB_exam_us',
                   'M200B1 BB_exam_ims',
                   'Combined_Scores']
# new df
m200_df.to_csv('data/m200_df_modified.csv', index=False)

KeyError: ('Last Name', 'First Name', 'Learner ID', 'Weighted Total', 'Total pts', 'M200P1 CTGL', 'M299A1 -div_mvmt_plan', 'M200B1 BB_exam_us', 'M200B1 BB_exam_ims', 'Combined_Scores')

In [ ]:
# Define a new list of column names
new_column_names = ['last_name',
                   'first_name',
                   'SG',
                   'ID'
                   'Wt_total', 
                   'Total_pts',
                   'CTGL',
                   'div_mvmt_plan',
                   'BB_exam_us',
                   'BB_exam_ims',
                   'Combined_Scores'
                   ]  
                    # Continue this list to match the number of columns in your DataFrame

# Assign the new column names to the DataFrame
m200_df.columns = new_column_names
